In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
def query(q):
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(q)
    rows = cur.fetchall()

    # pretty print the column names
    for k in rows[0].keys():
        print(k.ljust(28), end='')
    
    print()  # empty line

    # print a line under each column name
    for k in rows[0].keys():
        print(('-' * len(k)).ljust(28), end='')

    print('\n')  # double empty line

    for row in rows: 
        for element in row:
            print(str(element).ljust(28), end='')
        print()

In [3]:
conn=create_connection("sqlite_db_pythonsqlite.db")

2.6.0


### Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [4]:
query("SELECT * FROM(SELECT facility, SUM(session_cost) AS revenue FROM (SELECT name AS facility, CASE WHEN b.memid = 0 THEN (slots * guestcost) ELSE (slots * membercost) END AS session_cost FROM Bookings as b INNER JOIN Members as m ON b.memid = m.memid INNER JOIN Facilities as f ON b.facid = f.facid) as session_costs GROUP BY facility) as revenues WHERE revenue <1000 ORDER BY revenue")


facility                    revenue                     
--------                    -------                     

Table Tennis                180                         
Snooker Table               240                         
Pool Table                  270                         


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [5]:
query("SELECT m.surname ||', '|| m.firstname AS Member, r.surname ||', '  || r.firstname AS Recommended_By FROM Members AS m LEFT JOIN Members AS r ON m.recommendedby = r.memid ORDER BY Member")


Member                      Recommended_By              
------                      --------------              

Bader, Florence             Stibbons, Ponder            
Baker, Anne                 Stibbons, Ponder            
Baker, Timothy              Farrell, Jemima             
Boothe, Tim                 Rownam, Tim                 
Butters, Gerald             Smith, Darren               
Coplin, Joan                Baker, Timothy              
Crumpet, Erica              Smith, Tracy                
Dare, Nancy                 Joplette, Janice            
Farrell, David              None                        
Farrell, Jemima             None                        
GUEST, GUEST                None                        
Genting, Matthew            Butters, Gerald             
Hunt, John                  Purview, Millicent          
Jones, David                Joplette, Janice            
Jones, Douglas              Jones, David                
Joplette, Janice            Sm

### Q12: Find the facilities with their usage by member, but not guests

In [6]:
query("SELECT Member, name AS facility, SUM(slots) AS total_uses FROM (SELECT b.slots, f.name, m.surname ||', '|| m.firstname AS Member FROM `Bookings` AS b INNER JOIN Facilities AS f ON b.facid=f.facid INNER JOIN Members as m ON m.memid = b.memid WHERE b.memid != 0) AS member_bookings GROUP BY Member, facility")

Member                      facility                    total_uses                  
------                      --------                    ----------                  

Bader, Florence             Badminton Court             27                          
Bader, Florence             Massage Room 2              4                           
Bader, Florence             Pool Table                  23                          
Bader, Florence             Snooker Table               66                          
Bader, Florence             Squash Court                4                           
Bader, Florence             Table Tennis                86                          
Bader, Florence             Tennis Court 1              3                           
Bader, Florence             Tennis Court 2              24                          
Baker, Anne                 Badminton Court             30                          
Baker, Anne                 Massage Room 1              6       

### Q13: Find the facilities usage by month, but not guests

In [7]:
query("SELECT name AS facility, month, SUM(slots) as Slots_Used_by_Members FROM (SELECT b.slots, f.name, strftime('%m', starttime) AS month FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid=f.facid INNER JOIN Members as m ON m.memid = b.memid WHERE b.memid != 0) AS member_bookings GROUP BY facility, month")


facility                    month                       Slots_Used_by_Members       
--------                    -----                       ---------------------       

Badminton Court             07                          165                         
Badminton Court             08                          414                         
Badminton Court             09                          507                         
Massage Room 1              07                          166                         
Massage Room 1              08                          316                         
Massage Room 1              09                          402                         
Massage Room 2              07                          8                           
Massage Room 2              08                          18                          
Massage Room 2              09                          28                          
Pool Table                  07                          110     